## Get the data

Real-world data tends to occupy relational databases and Amazon S3 buckets. You better brush off your SQL and file processing skills. Luckily, languages like Python have a wide variety of helpers (both built-in and as external packages).

Working with raw files require an understanding of how text and image files are stored, can be loaded and shown.

### Start with a sample

Get 10 or 100 examples. Try to get somewhat of a representative subset of the data, but don't try too hard - a simple random subset will do (for now).

### Request more data

Some additional data can be easily stored for you. How easy would it be to get that done? Can someone else do it for you?

## Data from a relational database

One easy way to deal with data stored in a database is to use a tool to extract what you need and convert it to a Pandas data frame.

We'll use data provided by OSMI (Open Source Mental Illness) on mental health in the tech industry. It contains surveys on mental health disorders and their frequency through the industry. Let's download it:

In [19]:
!wget -q https://github.com/curiousily/Hackers-Guide-to-Deep-Learning/raw/master/data/mental_health.sqlite

The SQLite file contains data that is similar to what you might have in your production systems. Let's load it and see what tables it contains:

In [18]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

db_engine = create_engine("sqlite:///mental_health.sqlite")

db_engine.table_names()

['Answer', 'Question', 'Survey']

We can use Pandas to look at a sample of each data:

In [22]:
import pandas as pd

pd.read_sql("SELECT * FROM Answer LIMIT 5", db_engine)

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1


In [28]:
pd.read_sql("SELECT * FROM Question LIMIT 5", db_engine)

,questiontext,questionid
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5


In [27]:
pd.read_sql("SELECT * FROM Survey LIMIT 5", db_engine)

,SurveyID,Description
0,2014,mental health survey for 2014
1,2016,mental health survey for 2016
2,2017,mental health survey for 2017
3,2018,mental health survey for 2018
4,2019,mental health survey for 2019


Pretty straight forward, we have questions and answers from different people.

Ideally, we would want all of this into a single data frame. We can use a simple JOIN to get that. Note that we don't need the Survey table.

In [36]:
sql_statement = """
SELECT Answer.*, Question.questiontext
FROM Answer
INNER JOIN Question ON Answer.QuestionID=Question.questionid
"""
survey_df = pd.read_sql(sql_statement, db_engine)
survey_df.head()

,AnswerText,SurveyID,UserID,QuestionID,questiontext
0,37,2014,1,1,What is your age?
1,44,2014,2,1,What is your age?
2,32,2014,3,1,What is your age?
3,31,2014,4,1,What is your age?
4,31,2014,5,1,What is your age?


You can use this same technique for datasets with millions of rows to get a regular Pandas data frame and go from there.

## Text data

## Image data

## Labelling

Don't have enough or no data at all? You can create your own datasets! Yes, but that is not an easy task.

## Look at the data

## Is the data clean?

## Splitting

## Versioning